# <center>${\textbf{Top Potential Driver in Qualifying Session}}$</center>

${\textbf{Overview}}$

A Formula One season consists of a series of races, known as Grands Prix, which take place worldwide on purpose-built circuits and on public roads. <br>
A single competition takes place over three days, day one is a practice session ,the second day is the qualification round in which racers compete against time in order to obtain an advantageous position during  the third day which is the final race where drivers race against each other .<br>
Most importantly for our case a  qualifying session is held before each race to determine the order cars will be lined up in at the start of the race, with the fastest qualifier starting at the front and the slowest at the back.<br>
Currently, the first qualifying period (Q1) is eighteen minutes long, with all twenty cars competing.<br>
At the end of Q1, the five slowest drivers are eliminated from further qualification rounds, and fill positions sixteen to twenty on the grid based on their fastest lap time. Any driver attempting to set a qualifying time when the period ends is permitted to finish his lap, though no new laps may be started once the chequered flag is shown. <br>
After a short break, the second period (Q2) begins, with fifteen cars on the circuit.
<br> At the end of Q2, the five slowest drivers are once again eliminated, filling grid positions eleven to fifteen. Finally, the third qualifying period (Q3) features the ten fastest drivers from the second period. 
<br>The drivers are issued a new set of soft tyres and have twelve minutes to set a qualifying time, which will determine the top ten positions on the grid. The driver who sets the fastest qualifying time is said to be on pole position, the grid position that offers the best physical position from which to start the race.<br>

${\textbf{Importance of Data}}$

The insight analysis drills down to all information on the Formula 1 races, drivers, constructors, qualifying, circuits, lap times, pit stops, championships from 1950 till the latest 2021 season.<br>
With the amount of data being captured, analyzed and used to design, build and drive the Formula 1 cars is astounding. It is a global sport being followed by millions of people worldwide and it is very fascinating to see drivers pushing their limit in these vehicles to become the fastest racers in the world!<br>

${\textbf{Import Libraries}}$

In [153]:
%matplotlib inline
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pip
import geopandas
import altair as alt
from descartes.patch import PolygonPatch
from datetime import datetime
from apyori import apriori 
import plotly.express as px
from sklearn.svm import SVC
import warnings
warnings.simplefilter("ignore")

# importing ML libraries 
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
import pickle


## ${\textbf{Part 1 - First Steps  }}$

>${\textbf{Connect with Database}}$

Previously on the integration Phase we have used Talend Open Studio for Intergration Services linkes with a PostgresSQL .<br>So now it's only logical that we connect python with the postgres Database in order to retrieve our dimensional data

In [154]:
def DbConnect():
    conn = psycopg2.connect(host="localhost",database="FormulaOne",port=5432,user='postgres',password='root')
    return conn

>${\textbf{Get data from QualifyingFact and related tables}}$

In [155]:
weather = pd.read_sql('select * from "WeatherDim"', con=DbConnect())
date= pd.read_sql('select * from "DateTimeDim"', con=DbConnect())
circuit = pd.read_sql('select * from "CircuitDim"', con=DbConnect())
races= pd.read_sql('select * from "RaceDim"', con=DbConnect())
driver= pd.read_sql('select * from "DriverDim"', con=DbConnect())
constructor= pd.read_sql('select * from "ConstructorsDim"', con=DbConnect())
qualifyingFact= pd.read_sql('select * from "QualifyingFact"', con=DbConnect())

>${\textbf{Merging Data to a single DataFrame }}$

Renaming certain columns is essential for the merging 

In [156]:
races.rename(columns = {'datetime_fk':'dateId'}, inplace = True)
races.rename(columns = {'weather_fk':'weatherID'}, inplace = True)
races.rename(columns = {'circuit_fk':'CircuitId'}, inplace = True)
qualifyingFact.rename(columns = {'race_fk':'raceId'}, inplace = True)
qualifyingFact.rename(columns = {'driver_fk':'DriverID'}, inplace = True)
qualifyingFact.rename(columns = {'constructor_fk':'constructorId'}, inplace = True)

In [157]:
df1 = pd.merge(races,date, on='dateId', how='inner')
df2 = pd.merge(df1,circuit, on='CircuitId', how='inner')
df3 = pd.merge(df2,weather, on='weatherID', how='inner')
df4 = pd.merge(qualifyingFact,df3, on='raceId', how='inner')
df5 = pd.merge(df4,driver, on='DriverID', how='inner')
Fact = pd.merge(df5,constructor, on='constructorId', how='inner')

## ${\textbf{Part 2 - Exploratory Data Analysis  }}$

> ${\textbf{Renaming columns }}$

In [158]:
Fact.rename(columns = {'name_x':'nameGP','name_y':'nameCircuit','Nationality':'driverNationality'}, inplace = True)

> ${\textbf{Converting Date of birth to AGE}}$

The date of birth by itself can't be a great axis that we could retrieve and explain our  data from it so we opted for a conversion from date type to integer

In [159]:
Fact['Dob']=pd.to_datetime(Fact['Dob'])
date=datetime.today()-Fact['Dob']
Fact['age']=round(date.dt.days/365)

> ${\textbf{Concatenating firstname and lastname }}$

In [160]:
Fact["nameDriver"]=Fact["ForeName"]+" "+Fact["SurName"]

> ${\textbf{ Converting DateTime.Time Format to Milliseconds for a better overlook }}$

Various methods such as aggregation or summation can't work with dateTime.Time types , also previously we noticed a great deal of null data with is explained in the introduction so nulls will be zeros and time will be converted to milliseconds

In [161]:
Fact['q1_sec']=0
for i in range(0,len(Fact)):
    t=Fact['q1'].iat[i]
    if t is None:
        Fact['q1_sec'].iat[i] = int(0)
    else:
        Fact['q1_sec'].iat[i] = int(t.hour*3600000000+t.minute*60000000+t.second*1000000+t.microsecond)

In [162]:
Fact['q2_sec']=0
for i in range(0,len(Fact)):
    t=Fact['q2'].iat[i]
    if t is None:
        Fact['q2_sec'].iat[i] = int(0)
    else:
        Fact['q2_sec'].iat[i] = int(t.hour*3600000000+t.minute*60000000+t.second*1000000+t.microsecond)

In [163]:
Fact['q3_sec']=0
for i in range(0,len(Fact)):
    t=Fact['q3'].iat[i]
    if t is None:
        Fact['q3_sec'].iat[i] = int(0)
    else:
        Fact['q3_sec'].iat[i] = int(t.hour*3600000000+t.minute*60000000+t.second*1000000+t.microsecond)

> ${\textbf{Dropping unaffectful columns}}$

In [164]:
Fact.drop(columns=['raceId',
                   'DriverID',
                   'constructorId',
                   'CircuitId',
                   'dateId',
                   'weatherID',
                   'constructorRef',
                   'DriverRef',
                   'circuitRef',
                   'ForeName',
                   'SurName',
                   'Dob',
                   'number',
                   'time',
                   'Number',
                    'day',
                   'car',
                   'month',
                   'Altitude',
                   'location',
                   'points',
                   'img',
                   'weather',
                   'q1',
                   'q2',
                   'q3'],axis=1,inplace=True)

> ${\textbf{Columns }}$

In [165]:
Fact.columns

Index(['position', 'round', 'nameGP', 'date', 'year', 'nameCircuit',
       'latitude', 'longitude', 'country', 'Code', 'driverNationality',
       'constructorName', 'constructorNationality', 'age', 'nameDriver',
       'q1_sec', 'q2_sec', 'q3_sec'],
      dtype='object')

> ${\textbf{Data types and shape  }}$

In [166]:
Fact.info()
Fact.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4759 entries, 0 to 4758
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   position                4759 non-null   int64  
 1   round                   4759 non-null   int64  
 2   nameGP                  4759 non-null   object 
 3   date                    4759 non-null   object 
 4   year                    4759 non-null   int64  
 5   nameCircuit             4759 non-null   object 
 6   latitude                4759 non-null   float64
 7   longitude               4759 non-null   float64
 8   country                 4759 non-null   object 
 9   Code                    4759 non-null   object 
 10  driverNationality       4759 non-null   object 
 11  constructorName         4759 non-null   object 
 12  constructorNationality  4759 non-null   object 
 13  age                     4759 non-null   float64
 14  nameDriver              4759 non-null   

(4759, 18)

Overall we have 18 columns and 4759 rows

> ${\textbf{Head and tails }}$

In [167]:
Fact.head()

,position,round,nameGP,date,year,nameCircuit,latitude,longitude,country,Code,driverNationality,constructorName,constructorNationality,age,nameDriver,q1_sec,q2_sec,q3_sec
0,1,1,Australian Grand Prix,2008-03-16,2008,Albert Park Grand Prix Circuit,-37.84970,144.96800,Australia,HAM,British,McLaren,British,37.0,Lewis Hamilton,86572000,85187000,86714000
1,4,2,Malaysian Grand Prix,2008-03-23,2008,Sepang International Circuit,2.76083,101.73800,Malaysia,HAM,British,McLaren,British,37.0,Lewis Hamilton,95392000,94627000,96709000
2,3,3,Bahrain Grand Prix,2008-04-06,2008,Bahrain International Circuit,26.03250,50.51060,Bahrain,HAM,British,McLaren,British,37.0,Lewis Hamilton,92750000,91922000,93292000
3,5,4,Spanish Grand Prix,2008-04-27,2008,Circuit de Barcelona-Catalunya,41.57000,2.26111,Spain,HAM,British,McLaren,British,37.0,Lewis Hamilton,81366000,80825000,82096000
4,3,5,Turkish Grand Prix,2008-05-11,2008,Istanbul Park,40.95170,29.40500,Turkey,HAM,British,McLaren,British,37.0,Lewis Hamilton,86192000,86477000,87923000


In [168]:
Fact.tail()

,position,round,nameGP,date,year,nameCircuit,latitude,longitude,country,Code,driverNationality,constructorName,constructorNationality,age,nameDriver,q1_sec,q2_sec,q3_sec
4754,21,17,Japanese Grand Prix,2016-10-09,2016,Suzuka Circuit,34.8431,136.5410,Japan,OCO,French,Manor Marussia,British,26.0,Esteban Ocon,93353000,0,0
4755,22,18,United States Grand Prix,2016-10-23,2016,Circuit of the Americas,30.1328,-97.6411,USA,OCO,French,Manor Marussia,British,26.0,Esteban Ocon,98806000,0,0
4756,20,19,Mexican Grand Prix,2016-10-30,2016,Autódromo Hermanos Rodríguez,19.4042,-99.0907,Mexico,OCO,French,Manor Marussia,British,26.0,Esteban Ocon,81881000,0,0
4757,20,20,Brazilian Grand Prix,2016-11-13,2016,Autódromo José Carlos Pace,-23.7036,-46.6997,Brazil,OCO,French,Manor Marussia,British,26.0,Esteban Ocon,73432000,0,0
4758,20,21,Abu Dhabi Grand Prix,2016-11-27,2016,Yas Marina Circuit,24.4672,54.6031,UAE,OCO,French,Manor Marussia,British,26.0,Esteban Ocon,102286000,0,0


> ${\textbf{checking of null values }}$

As previously explained we have anticipated the nulls and fixed them in the preporcessing phase

In [169]:
Fact.isna().sum()

position                  0
round                     0
nameGP                    0
date                      0
year                      0
nameCircuit               0
latitude                  0
longitude                 0
country                   0
Code                      0
driverNationality         0
constructorName           0
constructorNationality    0
age                       0
nameDriver                0
q1_sec                    0
q2_sec                    0
q3_sec                    0
dtype: int64

In [170]:
selected_columns = Fact[["Code","position","age","q3_sec","driverNationality"]]
df = selected_columns.copy()


In [171]:
df['winning']=0
for i in range(0,len(df)):
    pos=df['position'].iat[i]
    if pos>0 and pos<6:
         df['winning'].iat[i] = int(1)
    else:
        
         df['winning'].iat[i] = int(0)

In [172]:
df.drop(columns=['position'],axis=1,inplace=True)

In [173]:
df1=df
df1=df1.groupby(["Code"]).sum('winning')

In [174]:
df1["potential"]=df1["winning"]/Fact.groupby(["Code"]).size()

In [175]:
df=df1["potential"].to_frame()

In [176]:
df.to_excel(r'potential.xlsx')

In [177]:
df1 = pd.merge(df,driver, on='Code', how='inner')
df1['Dob']=pd.to_datetime(df1['Dob'])
date=datetime.today()-df1['Dob']
df1['age']=round(date.dt.days/365)
df1.drop(columns=['DriverID','DriverRef','Number','ForeName','SurName','Dob','car'],axis=1,inplace=True)
df1['points'] = df1['points'].fillna(0).astype(np.int64, errors='ignore')
df1['age'] = df1['age'].fillna(0).astype(np.int64, errors='ignore')
df=df1

In [178]:
df.sort_values(by="potential",ascending=False).head(15)

,Code,potential,Nationality,points,age
13,HAM,0.872727,British,387,37
48,VET,0.665414,German,43,35
4,BOT,0.626506,Finnish,226,33
34,RAI,0.480645,Finnish,10,43
47,VER,0.456989,Dutch,395,25
46,VER,0.456989,French,395,32
36,ROS,0.446602,German,385,37
21,LEC,0.411765,Monegasque,159,25
2,ALO,0.392157,Spanish,81,41
25,MAS,0.334630,Brazilian,43,41


In [179]:
le = LabelEncoder()
df["Code"] = le.fit_transform(df["Code"])
df["Nationality"] = le.fit_transform(df["Nationality"])

In [180]:
df.sort_values(by="potential",ascending=False).head(15)

,Code,potential,Nationality,points,age
13,13,0.872727,4,387,37
48,46,0.665414,10,43,35
4,4,0.626506,8,226,33
34,33,0.480645,8,10,43
47,45,0.456989,7,395,25
46,45,0.456989,9,395,32
36,35,0.446602,10,385,37
21,20,0.411765,15,159,25
2,2,0.392157,19,81,41
25,24,0.334630,3,43,41


In [64]:
X = df.drop(['potential'], axis=1)
y = df[['potential']]
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=3)

In [65]:
# #Régression linéaire multiple (toutes les variables de Boston) 
# linreg2 = LinearRegression(fit_intercept= True, normalize=False) 
# #pour imposer une ordonnée à l'origine (Par défaut ='True')
# # Normalisation des données normalize=True
# linreg2.fit(X_train, y_train)
# y_pred = linreg2.predict(X_test)

# print('test_score = ',linreg2.score(X_test,y_test)) 
# print('R2 = ',r2_score(y_test, y_pred))
# print('MAE = ',mean_squared_error(y_test, y_pred))
# print('RMSE = ', np.sqrt(mean_squared_error(y_test, y_pred)))
# print('MAE = ', mean_absolute_error(y_test, y_pred))
# print('MeadianAE = ', median_absolute_error(y_test, y_pred))

# print('Intercept = ', linreg2.intercept_)
# print('Coefficients : ',linreg2.coef_)

regressor = LinearRegression()

#Fitting model with trainig data
regressor.fit(X, y)

LinearRegression()

In [66]:
test={'Code':['0'],'Nationality':['24'],'points':['170'],'age':['30']}
df=pd.DataFrame(test)
pred = regressor.predict(df)
print(pred)

[[0.25111984]]


In [32]:
pickle.dump(regressor, open('performancePredictor.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('performancePredictor.pkl','rb'))